In [2]:
library(tidyverse)

In [3]:
library(showtext)
font_add_google(name = "Black Han Sans", family = "blackhansans")
showtext_auto()

Loading required package: sysfonts

Loading required package: showtextdb



## 계층적 군집분석
- n개의 군집으로 시작해 점차 군집의 개수를 줄여 나가남

1. 완전연결법 (최대) : 군집과 군집 또는 데이터와의 거리 계산시 최장거리를 거리로 계산하여 산행렬 수정 (더 조밀한 군집)
2. 단인연결법 (최단) : 최소값을 거리로 간주 (길고 느슨한 군집)
3. 평균연결법 
4. 중심연결법 : 군집별 중심간의 비유사도를 계산
5. 와드연결법 (최소분산) : 전체 군집 내 분산을 최소화, 각 단계에서 군집간 거리가 가장 작은 군집쌍이 병합, 더 조밀한 군집 생성

- **범주형 변수들을군집화를 위한 거리계산에 사용하기 위하여 수치형으로 변환 (원-핫 인코딩)**
- 군집 분석은 관측치 간의 거리를 이용하기 때문에 변수의 단위가 결과에 큰 영향을 미침. 따라서 변수에 표준화를 적용 
- 단일값을 가지는 변수들은 군집화를 위한 데이터에서 제외

In [16]:
iris_scale <- iris %>% select(-Species) %>%
    scale() 

### dist() :
- ```data```
- ```method``` : "euclidean"`, "maximum", "manhattan", "canberra", "binary", "minkowski"

### ```hclust()```
- ```data```
- ```method``` : "single", "complete", "average", :median", "ward.D", "centroid", "mcquitty"

In [23]:
d <- dist(iris_scale, method = "euclidean")
hc1 <- hclust(d, method = "complete")

# plot(hc1)

### ```agnes()``` : 합병형(상향식, AGNES / 응집 군집화) 
### ```ac``` : 응집계수 (군집 구조의 강도)
- 값이 1에 가까울수록 균형 잡힌 군집 구조
- 값이 0에 가까울수록 균형이 좋지 않음
- n이 커질수록 ac가 커짐

In [25]:
library(cluster)

In [27]:
hc2 <- agnes(iris_scale, method = "complete")
hc2$ac 

[1] 0.9438858

### ```diana()``` : 분리형 (하향식, DIANA / 분할 계층적 군집화)
### ```dc``` : 분할 계수 (분할의 강도)
- 1에 가까울수록 분할이 더 강함

In [32]:
hc3 <- diana(iris_scale)
hc3$dc

[1] 0.9397208

#### 최적의 군집 결정

### ```fviz_nbclust()```
- ```FUN``` : hcut : 계층적 / kmeans : K-Means
- ```method``` : "wss", "silhouette", "gpp_stat"

In [34]:
library(factoextra)

In [ ]:
fviz_nbclust(iris_scale, 
             FUN = hcut, 
             method = "wss", 
             k.max = 10)  

#### 덴드로그램
### ```fviz_dend()```
- ```x``` : dendrogram, hclust, agnes, diana, hcut, hkmeans객체
- ```k``` : cut할 군집 수
- ```horiz``` : 가로세로 전환 (TRUE / FALSE)
- ```rect``` : cut한 군집에 사각형을 추가할지 여부
- ```rect_border``` : 사각형의 테두리 색상 및 선 유형.
- ```k_colors``` : 군집에 사용할 색상을 포함하는 벡터. (k개의 색상을 포함) [브루어 팔레트 : "RdBu", "Blues", "npg", "aaas", "lancet", "jco", "ucscgb", "uchicago", "simpsons" 및 "rickandmorty", "grey")]
- ```type``` : 플롯의 유형 ["rectangle", "circular"]

In [39]:
hc5 <- hclust(d, method = "ward.D2")

#### 원하는 수의 그룹으로 분할
### ```cutree()```

In [43]:
sub_grp <- cutree(hc5, k = 8)

In [ ]:
plot(hc5)
# fviz_dend(hc5)

In [54]:
# plot(hc5)
# rect.hclust(hc5, k = 8, border = "red")

### 비계층적 군집분석
1. 거리측정
- 유클리드 : 이상값에 매우 민감, 변수가 정규분포에 근사할때, 합리적
- 변수가 정규성에서 크게 벗어나거나, 기존 이상갑에 대해 robust하길 원한다면 맨하탄, 민코우스키, Gower거리 사용
- Gower 거리 : 범주형 및 순서형 특징을 포함시
- 코사인거리 : 텍스트 마이닝
- 상관기반 거리 : 관측값의 크기 차이는 크지만, 형태가 비슷한 un-scaled데이터일 경우, 비상관기반 거리측정보다 두 관찰간의 상관관계를 더 잘 포착

### K-평균 알고리즘
- 복잡한 기하학적 그룹을 포착하는데 비효율적

### ```kmeans()```
- ```centers```
- ```nstart```

```withinss``` : 군집내 변동
```tot.withinss``` : 군집 내 변동 총 제곱합
```centers``` : 각 군집의 중심

In [58]:
km <- kmeans(iris_scale, 4)

### ```fviz_nbclust()```
- ```FUN``` : hcut : 계층적 / kmeans : K-Means
- ```k.max```
- ```method``` : "wss", "silhouette", "gpp_stat"

#### 군집의 수
- k가 클수록 균질성을 향상시킬 수 있습니다. 그러나 과적합의 위험
- 주어진 사전적인 라벨값이 있다면, 라벨값을 k로 사용
- 사전적인 정보가 없다면 k = sqrt(n/2)

#### 엘보우 기법
- 군집내 총 제곱합(WSS)은 군집의 간결함의 정도, 가능한 최소화
- k별 WSS계산 => WSS값의 곡선을 plot => plot의 엘보우 포인트를 적절한 k로 선정

#### 변수의 수가 증가함에 따라 k-means의 성능이 저하
- 적절한 엘보우 포인트 탐색 불가 => Gower 거리 측정

#### Gower 거리
- 정량적 (간격) 데이터 : 정규화 맨해튼 거리
- 순서형 데이터 : 변수가 먼저 순위가 매겨진 다음, 맨하탄 거리가 동률에 대한 특별 조정과 함께 사용
- 명목형 데이터 : k개의 범주를 가진 범주형 변수는, k개의 열을 가진 데이터로 변환(원-핫 인코딩), 주사위 계수 (Dice coefficient)가 사용됨,
    - 주사위 계수 : 두 관측치(X, Y)에 대해 (모두 원-핫 인코딩된 범주형 데이터)
    - a = X와 Y가 모두 1인 dummy 수
    - b = X가 1, Y가 0인 dummy 수
    - c = X가 0, Y가 1인 dummy 수
    - d = X와 Y가 모두 0인 dummy 수
    - 
D = 2a / (2a + b + c)

### 스펙트럼 군집화
- 복잡한 기하학적 그룹을 포착하는데 효율적

In [60]:
library(kernlab)

In [ ]:
sc <- specc(iris_scale, centers = 2)
sc@.Data # 군집
sc@centers # 군집별 중심
sc@withinss # 군집내 변동

### ```cluster::daisy()```
- 데이터 세트의 관측치 사이의 모든 쌍별 비유사도(거리)를 계산
- metric : "gower"로 지정함으로써 Gower 거리 계산 ["euclidean", "manhattan", "gower"}

In [70]:
library(cluster)

In [73]:
gower_dist <- daisy(iris_scale, metric = "gower")

#### Gower 거리 행#렬은 ```cluster::diana(), cluster::agnes(), cluster::pam()```에만 사용 가능
#### ```cluster::kmeans(), cluster::clara()```에는 적용 불가

### 대체 분할 방법 (PAM)
- 데이터의 차원이 증가함에 따라 더 많은 이상치 발생 가능
- k -means는 평균을 사용하므로 이상치에 취약
- 이의 대안으로 PAM(partitioning around medians)을 사용, 이상치에 더 robust하게 만듬

In [75]:
pm <- pam(gower_dist, k = 8, diss = T)

### clustering large applications (CLARA)
- 데이터 세트가 계층적으로 커지면 k-means와 PAM 군집화가 느려짐
- 이의 대안으로 CLARA사용, 전체 데이터에 대해 중심점(medoid)를 탐색하는 대신 작은 크기에 대해 kmeans나 PAM을 적용
1. 데이터 세트를 고정 크기의 여러 하위집합으로 무작위로 분할.
2. 각 부분 집합에 대해 PAM 알고리즘을 계산하고, k-medoid에 해당하는 항목을 선택. 전체 데이터 세트의 각 관측값을 가장 가까운 medoid에 할당합니다.
3. 가장 가까운 medoid에 대한 관측치의 비유사성의 평균(또는 합)을 계산. 이것은 군집적합도의 척도로 사용.
4. 평균(또는 합계)이 최소인 하위 데이터 세트를 유지.

In [79]:
cl <- clara(iris_scale, k = 8)

### 모델 기반 군집화

### ```Mclust()```
- G : 군집수 (BIC를 계산할 혼합성분(군집)의 수를 지정), G를 지정하지 않으면, 자동으로 최적의 값 탐색
- modelNames : 군집화의 EM 단계에 적합할 모델을
    - 일변량 데이터의 경우(d=1): ["E", "V"]
    - 다변수 데이터의 경우(n>d): mclust.options("emModelNames")에서 사용 가능한 모든 모델
    - 다변수 데이터(n<=d): 나선형 및 대각선 모델, ["EII", "VII", "EEI", "EVI", "VEI", "VVI"]
    
### 혼합 분포 군집
- 데이터가 k개의 모수적 모형의 가중합으로 표현되는 모집단 모형으로부터 나왓다는 가정하에서 모수와 함꼐 가중치를 자료로부터 추정하는 방법을 사용
- K개의 각 모형은 군집을 의미하며, 각 데이터는 추정된 K개의 모형 중 어느 모형으로 나왓을 확률이 높은지에 따라 군집의 분류가 이루어짐
- 모수와 가중치의 최대가능도추정에는 EM알고리즘 사용

### EM 알고리즘
- E 단계 : 잠재변수 Z의 기대치 계산
- M 단계 : 잠재변수 Z의 기대치를 이용하여 파라미터 추정

#### 이상치에 민감, 군집을 몇 개의 모수로 표현 가능

In [82]:
library(mclust)

In [ ]:
mc <- Mclust(iris_scale, G = 4)
plot(mc, what = "density") # G개의 군집을 강조 표시하는 다변량 밀도 플롯
plot(mc, what = "uncertainty") 
# 어떤 군집의 구성원인지에 대한 불확실성이 높은 관측값을 강조하는 불확실성 플롯

- 확률적 군집 할당은, 군집 불확실성이 높거나 낮은 관측치를 식별하고 잠재적으로 고유하게 대상을 지정하거나 대체 솔루션을 제공할 수 있도록 하므로 매우 유용

In [98]:
mc

'Mclust' model object: (VEV,4) 

Available components: 
 [1] "call"           "data"           "modelName"      "n"             
 [5] "d"              "G"              "BIC"            "loglik"        
 [9] "df"             "bic"            "icl"            "hypvol"        
[13] "parameters"     "z"              "classification" "uncertainty"   

In [86]:
mc %>% summary

---------------------------------------------------- 
Gaussian finite mixture model fitted by EM algorithm 
---------------------------------------------------- 

Mclust VEV (ellipsoidal, equal shape) model with 4 components: 

 log-likelihood   n df       BIC       ICL
       -285.437 150 50 -821.4057 -837.1989

Clustering table:
 1  2  3  4 
26 24 34 66 

- 불확실성이 가장 높은 관측값 (불확실성 : 다른 군집에도 할당될 확률)

In [100]:
sort(mc$uncertainty, decreasing = T) %>% head %>% print

[1] 0.4721437 0.4481100 0.4103222 0.3976437 0.3479466 0.3379856


### 공분산 유형
- 공분산 행렬은 군집의의 볼륨(양), 모양 및 방향
- 군집과 밀도는, 크기와 모양이 비례

#### 공분산 행렬에 제약조건을 추가, 제약조건은 다음중 하나**
1. 볼륨: 각 군집은 거의 동일한 수의 관측치가 있다
2. 모양: 분포가 구형이 되도록 각 군집은 거의 동일한 분산을 가짐.
3. 방향: 각 군집은 강제로 축 정렬됨.

#### BIC기준 공분산 매개변수 식별, 최적의 군집수 탐색


In [101]:
opt_mc <- Mclust(iris_scale)

In [102]:
opt_mc

'Mclust' model object: (VVV,2) 

Available components: 
 [1] "call"           "data"           "modelName"      "n"             
 [5] "d"              "G"              "BIC"            "loglik"        
 [9] "df"             "bic"            "icl"            "hypvol"        
[13] "parameters"     "z"              "classification" "uncertainty"   

최적모델은 2개의 군집, BIC값이 가장 낮은 VVV 공분산 매개변수를 사용